In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np
import matplotlib as mpl

import matplotlib.pyplot as plt
import platform
if platform.system() == "Windows":
    plt.rcParams['font.family'] = ['Times New Roman',] # windows->Heiti TC
else:
    plt.rcParams['font.family'] = ['Heiti TC',] # windows->Heiti TC
    
plt.rcParams['axes.unicode_minus'] = False # windows->simhei
plt.rcParams['figure.dpi'] = 200

import geopandas as gpd
import shapely
from shapely.geometry import Point,Polygon,LineString,MultiLineString,MultiPoint,MultiPolygon

import pickle
from tqdm import tqdm
from glob import glob
import joblib

In [2]:
import platform
if platform.system() == "Windows":
    os.chdir(r'H:\BaiduSyncdisk\DR.MENG-Full\Y2024-002-DT-NANJING-ACCESSIBILITY-MAUP')
else:
    os.chdir(r'/Volumes/SANDISK/DR.MENG')

# GRIDS

In [3]:
data = joblib.load('./data_diff/plot_dist.joblib')
filter_50m_grid = gpd.read_file('./data_factors/filter_50m_gird.shp')
filter_50m_grid = filter_50m_grid.to_crs(32649)

In [4]:
data[0]['name']

'od_walk_grid250_od_900'

In [5]:
# [d['name'] for d in data]

In [6]:
name = data[0]['name']

In [7]:
data[0].keys()

dict_keys(['name', 'dist'])

In [8]:
# data[0]['original_5_95'].iloc[:10].explore()

In [9]:
# filter_50m_grid

In [10]:
grid50 = data[0]['dist'] # 这个是50米网格，获取其质心
grid50 = grid50.to_crs(32649)

In [11]:
# data[0]['name']对应的网格

In [12]:
level = data[0]['name'].split('grid')[-1].split('_')[0]

In [13]:
grid_larger = gpd.read_file(f'./shp/use_bounds/nanjing_grid{level}.shp')
grid_larger = grid_larger.to_crs(32649)

In [14]:
grid_larger['grid_larger_x'] = grid_larger['geometry'].representative_point().x
grid_larger['grid_larger_y'] = grid_larger['geometry'].representative_point().y

grid_largerx = grid_larger[['geometry','grid_larger_x','grid_larger_y']]

In [15]:
grid_largerx # 这个是250米网格，获取其质心

,geometry,grid_larger_x,grid_larger_y
0,"POLYGON ((1236836.200 3553205.992, 1236836.200...",1.236961e+06,3.553331e+06
1,"POLYGON ((1237086.200 3553205.992, 1237086.200...",1.237211e+06,3.553331e+06
2,"POLYGON ((1237336.200 3553205.992, 1237336.200...",1.237461e+06,3.553331e+06
3,"POLYGON ((1237586.200 3553205.992, 1237586.200...",1.237711e+06,3.553331e+06
4,"POLYGON ((1237836.200 3553205.992, 1237836.200...",1.237961e+06,3.553331e+06
...,...,...,...
13710,"POLYGON ((1231336.200 3595205.992, 1231336.200...",1.231461e+06,3.595331e+06
13711,"POLYGON ((1230586.200 3595455.992, 1230586.200...",1.230711e+06,3.595581e+06
13712,"POLYGON ((1230836.200 3595455.992, 1230836.200...",1.230961e+06,3.595581e+06
13713,"POLYGON ((1231086.200 3595455.992, 1231086.200...",1.231211e+06,3.595581e+06


In [16]:
grid_largerx_coord = gpd.sjoin(filter_50m_grid, grid_largerx, predicate='within')[['net_id','grid_larger_x','grid_larger_y']]

In [17]:
grid_largerx_coord

,net_id,grid_larger_x,grid_larger_y
1,1128598,1.239461e+06,3.553581e+06
2,1128599,1.239461e+06,3.553581e+06
3,1128600,1.239461e+06,3.553581e+06
4,1128601,1.239461e+06,3.553581e+06
6,1129450,1.239461e+06,3.553581e+06
...,...,...,...
69678,2152680,1.229211e+06,3.594831e+06
69681,2153431,1.229211e+06,3.594831e+06
69682,2153432,1.229211e+06,3.594831e+06
69683,2153433,1.229211e+06,3.594831e+06


# 公园数据

In [18]:
parks = joblib.load('./od_request_data_buildings/parks_with_id.joblib')

In [19]:
parks2 = gpd.read_file('./parks/裁剪后/裁剪后.shp')

In [20]:
parks2['name'] = parks2['name'].str.replace('南京市','')
parks2 = parks2.to_crs(32649)
parks2['area_clip'] = (parks2['geometry'].area / 10000).round(2)
parks2 = parks2.to_crs(4326)
parks2 = parks2[['name','area_clip']]

In [21]:
parks2

,name,area_clip
0,绣球公园,6.30
1,宝船厂遗址公园,8.77
2,乌龙潭公园,4.40
3,鼓楼公园,0.80
4,阅江楼景区,24.43
...,...,...
137,启龙亲江乐园,12.08
138,邓廷桢文化公园,1.89
139,小牛头山公园,7.57
140,NaN,0.27


In [22]:
parks = parks.to_crs(32649)
parks['area'] = (parks['geometry'].area / 10000).round(2)

In [23]:
parks = parks.merge(parks2,how='left')

In [24]:
def get_final_area(x):
    if np.isnan(x[1]):
        return x[0]
    else:
        return x[1]

In [25]:
parks['area'] = parks[['area','area_clip']].apply(get_final_area, axis=1)

In [26]:
parks.sort_values(by='area')

,name,geometry,entrance,park_id,area,area_clip
106,大钟亭公园,"POLYGON ((1235266.128 3574004.442, 1235288.578...","[[118.77925931852411, 32.06226721731419], [118...",p_105,0.40,0.40
42,城东公园,"POLYGON ((1243540.418 3563165.624, 1243541.674...","[[118.85691229767663, 31.960552138076586], [11...",p_42,0.47,0.47
2,鼓楼公园,"POLYGON ((1235072.273 3573800.305, 1235088.807...","[[118.77634680909901, 32.060983902760526]]",p_2,0.80,0.80
14,窨子山文化遗址公园,"POLYGON ((1237468.417 3566158.055, 1237467.170...","[[118.7970079250168, 31.99140213205]]",p_14,1.01,1.01
17,西安门遗址公园,"POLYGON ((1237417.065 3571845.110, 1237445.470...","[[118.8000213280367, 32.042198600387145], [118...",p_17,1.03,1.03
...,...,...,...,...,...,...
45,城北桥公园,"POLYGON ((1244689.465 3557802.571, 1244719.196...","[[118.86567132067046, 31.911988485068353], [11...",p_45,297.31,297.31
107,幕燕风景名胜区,"POLYGON ((1234368.428 3579962.400, 1234356.944...","[[118.78244570925843, 32.12821725884642], [118...",p_106,298.74,298.74
68,矿山公园,"POLYGON ((1248389.335 3625868.003, 1248306.860...","[[118.95222594596851, 32.496232332231536], [11...",p_68,379.96,379.96
28,将军山公园,"POLYGON ((1234864.663 3562457.963, 1234980.960...","[[118.77015166153492, 31.944829059470823], [11...",p_28,574.83,574.83


In [27]:
parks.loc[57,'area'] = 500

In [28]:
parks.sort_values(by='area')

,name,geometry,entrance,park_id,area,area_clip
106,大钟亭公园,"POLYGON ((1235266.128 3574004.442, 1235288.578...","[[118.77925931852411, 32.06226721731419], [118...",p_105,0.40,0.40
42,城东公园,"POLYGON ((1243540.418 3563165.624, 1243541.674...","[[118.85691229767663, 31.960552138076586], [11...",p_42,0.47,0.47
2,鼓楼公园,"POLYGON ((1235072.273 3573800.305, 1235088.807...","[[118.77634680909901, 32.060983902760526]]",p_2,0.80,0.80
14,窨子山文化遗址公园,"POLYGON ((1237468.417 3566158.055, 1237467.170...","[[118.7970079250168, 31.99140213205]]",p_14,1.01,1.01
17,西安门遗址公园,"POLYGON ((1237417.065 3571845.110, 1237445.470...","[[118.8000213280367, 32.042198600387145], [118...",p_17,1.03,1.03
...,...,...,...,...,...,...
45,城北桥公园,"POLYGON ((1244689.465 3557802.571, 1244719.196...","[[118.86567132067046, 31.911988485068353], [11...",p_45,297.31,297.31
107,幕燕风景名胜区,"POLYGON ((1234368.428 3579962.400, 1234356.944...","[[118.78244570925843, 32.12821725884642], [118...",p_106,298.74,298.74
68,矿山公园,"POLYGON ((1248389.335 3625868.003, 1248306.860...","[[118.95222594596851, 32.496232332231536], [11...",p_68,379.96,379.96
57,汤山地质公园,"POLYGON ((1259210.699 3578158.115, 1259287.294...","[[119.04787718233933, 32.0573319193085], [119....",p_57,500.00,1264.33


In [29]:
parks_df = pd.DataFrame({
                'name':parks['name'].repeat([len(i) for i in parks['entrance']]),
                'area':parks['area'].repeat([len(i) for i in parks['entrance']]),
                'park_id':parks['park_id'].repeat([len(i) for i in parks['entrance']]),
                'entrance':parks['entrance'].explode(),
              })

In [30]:
parks_df

,name,area,park_id,entrance
0,宝船厂遗址公园,8.77,p_0,"[118.73099014005813, 32.063971884943506]"
1,乌龙潭公园,4.40,p_1,"[118.7626038071835, 32.049716490117135]"
1,乌龙潭公园,4.40,p_1,"[118.76174980256653, 32.04939982947426]"
1,乌龙潭公园,4.40,p_1,"[118.76225895173785, 32.04957847965279]"
1,乌龙潭公园,4.40,p_1,"[118.75923799835127, 32.046188127755705]"
...,...,...,...,...
139,瑞龙郊野公园,35.55,p_137,"[118.6621181533641, 32.110792675250394]"
139,瑞龙郊野公园,35.55,p_137,"[118.66572473679064, 32.1084712510619]"
139,瑞龙郊野公园,35.55,p_137,"[118.66312933884412, 32.11036578205247]"
140,邓廷桢文化公园,1.89,p_138,"[118.9274261563141, 32.09197732070501]"


In [31]:
geometry = gpd.points_from_xy(parks_df['entrance'].str.get(0), parks_df['entrance'].str.get(1))
parks_df = gpd.GeoDataFrame(parks_df[['name','area','park_id']],
                           geometry=geometry, crs='4326')

parks_df = parks_df.to_crs(32649)

In [32]:
parks_df['park_x'] = parks_df['geometry'].x
parks_df['park_y'] = parks_df['geometry'].y

In [33]:
parks_df.head()

,name,area,park_id,geometry,park_x,park_y
0,宝船厂遗址公园,8.77,p_0,POINT (1230697.737 3573794.742),1.230698e+06,3.573795e+06
1,乌龙潭公园,4.40,p_1,POINT (1233808.488 3572424.640),1.233808e+06,3.572425e+06
1,乌龙潭公园,4.40,p_1,POINT (1233730.091 3572383.543),1.233730e+06,3.572384e+06
1,乌龙潭公园,4.40,p_1,POINT (1233776.912 3572406.916),1.233777e+06,3.572407e+06
1,乌龙潭公园,4.40,p_1,POINT (1233517.855 3572008.897),1.233518e+06,3.572009e+06


# s1：找:2km阈值范围内的的公园数量

In [34]:
grid50['gird50_x'] = grid50['geometry'].representative_point().x
grid50['gird50_y'] = grid50['geometry'].representative_point().y

In [35]:
grid50.head()

,net_id,Id,geometry,access_val_buildings,access_val_grid,diff_building_grid,gird50_x,gird50_y
0,1128597,south,"POLYGON ((1239336.200 3553505.992, 1239336.200...",0.0,0.0,0.0,1.239361e+06,3.553531e+06
1,1128598,south,"POLYGON ((1239386.200 3553505.992, 1239386.200...",0.0,0.0,0.0,1.239411e+06,3.553531e+06
2,1128599,south,"POLYGON ((1239436.200 3553505.992, 1239436.200...",0.0,0.0,0.0,1.239461e+06,3.553531e+06
3,1128600,south,"POLYGON ((1239486.200 3553505.992, 1239486.200...",0.0,0.0,0.0,1.239511e+06,3.553531e+06
4,1128601,south,"POLYGON ((1239536.200 3553505.992, 1239536.200...",0.0,0.0,0.0,1.239561e+06,3.553531e+06


In [36]:
parks_df.head()

,name,area,park_id,geometry,park_x,park_y
0,宝船厂遗址公园,8.77,p_0,POINT (1230697.737 3573794.742),1.230698e+06,3.573795e+06
1,乌龙潭公园,4.40,p_1,POINT (1233808.488 3572424.640),1.233808e+06,3.572425e+06
1,乌龙潭公园,4.40,p_1,POINT (1233730.091 3572383.543),1.233730e+06,3.572384e+06
1,乌龙潭公园,4.40,p_1,POINT (1233776.912 3572406.916),1.233777e+06,3.572407e+06
1,乌龙潭公园,4.40,p_1,POINT (1233517.855 3572008.897),1.233518e+06,3.572009e+06


In [37]:
grid50x = grid50.copy()

In [38]:
grid50x['geometry'] = grid50x['geometry'].buffer(2000)

In [39]:
result = gpd.sjoin(grid50x, parks_df, predicate='contains')

In [40]:
result = result.drop_duplicates(subset=['net_id', 'name'])

In [41]:
len(result)

174758

In [42]:
result.head()

,net_id,Id,geometry,access_val_buildings,access_val_grid,diff_building_grid,gird50_x,gird50_y,index_right,name,area,park_id,park_x,park_y
104,1175274,south,"POLYGON ((1240186.200 3554155.992, 1239990.166...",0.0,0.0,0.0,1.240211e+06,3.556181e+06,53,九龙湖公园,12.97,p_53,1.239119e+06,3.557856e+06
105,1175275,south,"POLYGON ((1240236.200 3554155.992, 1240040.166...",0.0,0.0,0.0,1.240261e+06,3.556181e+06,53,九龙湖公园,12.97,p_53,1.239119e+06,3.557856e+06
116,1176132,south,"POLYGON ((1240136.200 3554205.992, 1239940.166...",0.0,0.0,0.0,1.240161e+06,3.556231e+06,53,九龙湖公园,12.97,p_53,1.239119e+06,3.557856e+06
117,1176133,south,"POLYGON ((1240186.200 3554205.992, 1239990.166...",0.0,0.0,0.0,1.240211e+06,3.556231e+06,53,九龙湖公园,12.97,p_53,1.239119e+06,3.557856e+06
118,1176134,south,"POLYGON ((1240236.200 3554205.992, 1240040.166...",0.0,0.0,0.0,1.240261e+06,3.556231e+06,53,九龙湖公园,12.97,p_53,1.239119e+06,3.557856e+06


In [43]:
result.columns

Index(['net_id', 'Id', 'geometry', 'access_val_buildings', 'access_val_grid',
       'diff_building_grid', 'gird50_x', 'gird50_y', 'index_right', 'name',
       'area', 'park_id', 'park_x', 'park_y'],
      dtype='object')

In [44]:
len(result)

174758

In [45]:
# result.head().explore()

In [46]:
result_vl = result[['net_id', 'diff_building_grid', 'gird50_x', 'gird50_y', 'name', 'area', 'park_id', 'park_x', 'park_y']]

In [47]:
result_vl.head(1)

,net_id,diff_building_grid,gird50_x,gird50_y,name,area,park_id,park_x,park_y
104,1175274,0.0,1.240211e+06,3.556181e+06,九龙湖公园,12.97,p_53,1.239119e+06,3.557856e+06


In [48]:
len(result_vl)

174758

In [49]:
grid50_data = grid50[['net_id','geometry']].merge(result_vl, on='net_id', how='right')

In [50]:
grid50_data # 下一步是获取大网格的

,net_id,geometry,diff_building_grid,gird50_x,gird50_y,name,area,park_id,park_x,park_y
0,1175274,"POLYGON ((1240186.200 3556155.992, 1240186.200...",0.0,1.240211e+06,3.556181e+06,九龙湖公园,12.97,p_53,1.239119e+06,3.557856e+06
1,1175275,"POLYGON ((1240236.200 3556155.992, 1240236.200...",0.0,1.240261e+06,3.556181e+06,九龙湖公园,12.97,p_53,1.239119e+06,3.557856e+06
2,1176132,"POLYGON ((1240136.200 3556205.992, 1240136.200...",0.0,1.240161e+06,3.556231e+06,九龙湖公园,12.97,p_53,1.239119e+06,3.557856e+06
3,1176133,"POLYGON ((1240186.200 3556205.992, 1240186.200...",0.0,1.240211e+06,3.556231e+06,九龙湖公园,12.97,p_53,1.239119e+06,3.557856e+06
4,1176134,"POLYGON ((1240236.200 3556205.992, 1240236.200...",0.0,1.240261e+06,3.556231e+06,九龙湖公园,12.97,p_53,1.239119e+06,3.557856e+06
...,...,...,...,...,...,...,...,...,...,...
174753,2149713,"POLYGON ((1231186.200 3594505.992, 1231186.200...",0.0,1.231211e+06,3.594531e+06,平顶山公园,4.68,p_86,1.232440e+06,3.593442e+06
174754,2149714,"POLYGON ((1231236.200 3594505.992, 1231236.200...",0.0,1.231261e+06,3.594531e+06,平顶山公园,4.68,p_86,1.232440e+06,3.593442e+06
174755,2149715,"POLYGON ((1231286.200 3594505.992, 1231286.200...",0.0,1.231311e+06,3.594531e+06,平顶山公园,4.68,p_86,1.232440e+06,3.593442e+06
174756,2149719,"POLYGON ((1231486.200 3594505.992, 1231486.200...",0.0,1.231511e+06,3.594531e+06,平顶山公园,4.68,p_86,1.232440e+06,3.593442e+06


In [51]:
grid_largerx_coord

,net_id,grid_larger_x,grid_larger_y
1,1128598,1.239461e+06,3.553581e+06
2,1128599,1.239461e+06,3.553581e+06
3,1128600,1.239461e+06,3.553581e+06
4,1128601,1.239461e+06,3.553581e+06
6,1129450,1.239461e+06,3.553581e+06
...,...,...,...
69678,2152680,1.229211e+06,3.594831e+06
69681,2153431,1.229211e+06,3.594831e+06
69682,2153432,1.229211e+06,3.594831e+06
69683,2153433,1.229211e+06,3.594831e+06


In [52]:
result_vlx = grid50_data.merge(grid_largerx_coord, on='net_id')

In [53]:
result_vlx

,net_id,geometry,diff_building_grid,gird50_x,gird50_y,name,area,park_id,park_x,park_y,grid_larger_x,grid_larger_y
0,1176133,"POLYGON ((1240186.200 3556205.992, 1240186.200...",0.0,1.240211e+06,3.556231e+06,九龙湖公园,12.97,p_53,1.239119e+06,3.557856e+06,1.240211e+06,3.556331e+06
1,1176990,"POLYGON ((1240136.200 3556255.992, 1240136.200...",0.0,1.240161e+06,3.556281e+06,九龙湖公园,12.97,p_53,1.239119e+06,3.557856e+06,1.240211e+06,3.556331e+06
2,1176991,"POLYGON ((1240186.200 3556255.992, 1240186.200...",0.0,1.240211e+06,3.556281e+06,九龙湖公园,12.97,p_53,1.239119e+06,3.557856e+06,1.240211e+06,3.556331e+06
3,1176992,"POLYGON ((1240236.200 3556255.992, 1240236.200...",0.0,1.240261e+06,3.556281e+06,九龙湖公园,12.97,p_53,1.239119e+06,3.557856e+06,1.240211e+06,3.556331e+06
4,1176993,"POLYGON ((1240286.200 3556255.992, 1240286.200...",0.0,1.240311e+06,3.556281e+06,九龙湖公园,12.97,p_53,1.239119e+06,3.557856e+06,1.240211e+06,3.556331e+06
...,...,...,...,...,...,...,...,...,...,...,...,...
102867,2149712,"POLYGON ((1231136.200 3594505.992, 1231136.200...",0.0,1.231161e+06,3.594531e+06,平顶山公园,4.68,p_86,1.232440e+06,3.593442e+06,1.231211e+06,3.594581e+06
102868,2149713,"POLYGON ((1231186.200 3594505.992, 1231186.200...",0.0,1.231211e+06,3.594531e+06,平顶山公园,4.68,p_86,1.232440e+06,3.593442e+06,1.231211e+06,3.594581e+06
102869,2149714,"POLYGON ((1231236.200 3594505.992, 1231236.200...",0.0,1.231261e+06,3.594531e+06,平顶山公园,4.68,p_86,1.232440e+06,3.593442e+06,1.231211e+06,3.594581e+06
102870,2149719,"POLYGON ((1231486.200 3594505.992, 1231486.200...",0.0,1.231511e+06,3.594531e+06,平顶山公园,4.68,p_86,1.232440e+06,3.593442e+06,1.231461e+06,3.594581e+06


In [54]:
result_vlx2 = result_vlx[result_vlx['diff_building_grid'] != 0]

In [55]:
result_vlx2

,net_id,geometry,diff_building_grid,gird50_x,gird50_y,name,area,park_id,park_x,park_y,grid_larger_x,grid_larger_y
153,1196788,"POLYGON ((1238086.200 3557405.992, 1238086.200...",1.016043,1.238111e+06,3.557431e+06,九龙湖公园,12.97,p_53,1.239119e+06,3.557856e+06,1.238211e+06,3.557331e+06
158,1197658,"POLYGON ((1239786.200 3557455.992, 1239786.200...",1.093748,1.239811e+06,3.557481e+06,九龙湖公园,12.97,p_53,1.239119e+06,3.557856e+06,1.239711e+06,3.557581e+06
165,1198486,"POLYGON ((1239636.200 3557505.992, 1239636.200...",2.837656,1.239661e+06,3.557531e+06,九龙湖公园,12.97,p_53,1.239119e+06,3.557856e+06,1.239711e+06,3.557581e+06
178,1199280,"POLYGON ((1238086.200 3557555.992, 1238086.200...",2.032086,1.238111e+06,3.557581e+06,九龙湖公园,12.97,p_53,1.239119e+06,3.557856e+06,1.238211e+06,3.557581e+06
179,1199311,"POLYGON ((1239636.200 3557555.992, 1239636.200...",-0.864001,1.239661e+06,3.557581e+06,九龙湖公园,12.97,p_53,1.239119e+06,3.557856e+06,1.239711e+06,3.557581e+06
...,...,...,...,...,...,...,...,...,...,...,...,...
102766,2141538,"POLYGON ((1232236.200 3593905.992, 1232236.200...",0.036000,1.232261e+06,3.593931e+06,平顶山公园,4.68,p_86,1.232440e+06,3.593442e+06,1.232211e+06,3.593831e+06
102767,2141540,"POLYGON ((1232336.200 3593905.992, 1232336.200...",0.396145,1.232361e+06,3.593931e+06,平顶山公园,4.68,p_86,1.232440e+06,3.593442e+06,1.232461e+06,3.593831e+06
102768,2141541,"POLYGON ((1232386.200 3593905.992, 1232386.200...",0.458287,1.232411e+06,3.593931e+06,平顶山公园,4.68,p_86,1.232440e+06,3.593442e+06,1.232461e+06,3.593831e+06
102783,2142858,"POLYGON ((1232236.200 3594005.992, 1232236.200...",-0.159028,1.232261e+06,3.594031e+06,平顶山公园,4.68,p_86,1.232440e+06,3.593442e+06,1.232211e+06,3.594081e+06


# angle and distance

In [56]:
xxx = {
    'gird50_x':0,
    'gird50_y':0,
    'park_x':0,
    'park_y':1,
    'grid_larger_x': np.sqrt(3),
    'grid_larger_y': 0,
      }

In [57]:
import math

In [58]:
def get_angle(x):

    a = LineString([(x['grid_larger_x'], x['grid_larger_y']),(x['gird50_x'], x['gird50_y'])])
    b = LineString([(x['park_x'], x['park_y']),(x['gird50_x'], x['gird50_y'])])
    c = LineString([(x['grid_larger_x'], x['grid_larger_y']),(x['park_x'], x['park_y'])])

    a = round(a.length,3)
    b = round(b.length,3)
    c = round(c.length,3)
    # print(a,b,c)
    
    try:
        if a == 0: # 说明大小网格重合了
            return 0, a, b, c
        elif b == 0: # 说明小网格与公园重合了
            return np.nan, a, b, c
        else:
            cos_B = (a**2 + c**2 - b**2) / (2 * a * c)
            B = math.degrees(math.acos(cos_B))
            return round(B, 2), a, b, c
    except:
        return 0, a, b, c
    ##########################################
    # cos_C = (a**2 + b**2 - c**2) / (2 * a * b)
    # C = math.degrees(math.acos(cos_C))
    # # 计算角度A
    # cos_A = (b**2 + c**2 - a**2) / (2 * b * c)
    # A = math.degrees(math.acos(cos_A))
    # # 计算角度B
    # B = 180 - A - C  # 利用三角形内角和为180度的性质
    ##########################################

In [59]:
b = get_angle(xxx)
b

(30.0, 1.732, 1.0, 2.0)

In [60]:
result_vlx['data'] = result_vlx.apply(get_angle, axis=1)

In [61]:
sample = result_vlx.iloc[627]
sample

net_id                                                          1216504
geometry              POLYGON ((1238386.2003333508 3558755.991776327...
diff_building_grid                                             2.806058
gird50_x                                                 1238411.200333
gird50_y                                                 3558780.991776
name                                                       凤凰台公园（百家湖广场）
area                                                                6.0
park_id                                                            p_52
park_x                                                   1239727.761872
park_y                                                   3560226.417928
grid_larger_x                                            1238461.200333
grid_larger_y                                            3558830.991776
data                               (177.22, 70.711, 1955.145, 1884.514)
Name: 627, dtype: object

In [62]:
from pyproj import Proj, transform

# 定义两个坐标系
proj_src = Proj(init='epsg:32649')
proj_dst = Proj(init='epsg:4326')

# 假设你有一个点的坐标 (x, y)，这里以一个示例点为例
x_src, y_src = sample['gird50_x'], sample['gird50_y']  # 这只是一个示例坐标，实际使用时请替换为真实坐标
# 进行坐标转换
x_dst, y_dst = transform(proj_src, proj_dst, x_src, y_src)
print(f"转换后的坐标为: [{x_dst}, {y_dst}]")

转换后的坐标为: [118.8005167597025, 31.924784810234268]


In [63]:
# 假设你有一个点的坐标 (x, y)，这里以一个示例点为例
x_src, y_src = sample['park_x'], sample['park_y']  # 这只是一个示例坐标，实际使用时请替换为真实坐标
# 进行坐标转换
x_dst, y_dst = transform(proj_src, proj_dst, x_src, y_src)
print(f"转换后的坐标为: [{x_dst}, {y_dst}]")

转换后的坐标为: [118.81541393504081, 31.936850516193406]


In [64]:
# 假设你有一个点的坐标 (x, y)，这里以一个示例点为例
x_src, y_src = sample['grid_larger_x'], sample['grid_larger_y']  # 这只是一个示例坐标，实际使用时请替换为真实坐标
# 进行坐标转换
x_dst, y_dst = transform(proj_src, proj_dst, x_src, y_src)
print(f"转换后的坐标为: [{x_dst}, {y_dst}]")

转换后的坐标为: [118.8010787329154, 31.925199338794197]


In [65]:
result_vlx['angle'] = result_vlx['data'].str.get(0)
result_vlx['50_larger'] = result_vlx['data'].str.get(1)
result_vlx['50_park'] = result_vlx['data'].str.get(2)
result_vlx['larger_park'] = result_vlx['data'].str.get(3)

In [66]:
result_vlx

,net_id,geometry,diff_building_grid,gird50_x,gird50_y,name,area,park_id,park_x,park_y,grid_larger_x,grid_larger_y,data,angle,50_larger,50_park,larger_park
0,1176133,"POLYGON ((1240186.200 3556205.992, 1240186.200...",0.0,1.240211e+06,3.556231e+06,九龙湖公园,12.97,p_53,1.239119e+06,3.557856e+06,1.240211e+06,3.556331e+06,"(144.38, 100.0, 1957.807, 1875.651)",144.38,100.000,1957.807,1875.651
1,1176990,"POLYGON ((1240136.200 3556255.992, 1240136.200...",0.0,1.240161e+06,3.556281e+06,九龙湖公园,12.97,p_53,1.239119e+06,3.557856e+06,1.240211e+06,3.556331e+06,"(99.38, 70.711, 1888.464, 1875.651)",99.38,70.711,1888.464,1875.651
2,1176991,"POLYGON ((1240186.200 3556255.992, 1240186.200...",0.0,1.240211e+06,3.556281e+06,九龙湖公园,12.97,p_53,1.239119e+06,3.557856e+06,1.240211e+06,3.556331e+06,"(144.38, 50.0, 1916.517, 1875.651)",144.38,50.000,1916.517,1875.651
3,1176992,"POLYGON ((1240236.200 3556255.992, 1240236.200...",0.0,1.240261e+06,3.556281e+06,九龙湖公园,12.97,p_53,1.239119e+06,3.557856e+06,1.240211e+06,3.556331e+06,"(170.62, 70.711, 1945.451, 1875.651)",170.62,70.711,1945.451,1875.651
4,1176993,"POLYGON ((1240286.200 3556255.992, 1240286.200...",0.0,1.240311e+06,3.556281e+06,九龙湖公园,12.97,p_53,1.239119e+06,3.557856e+06,1.240211e+06,3.556331e+06,"(152.19, 111.803, 1975.226, 1875.651)",152.19,111.803,1975.226,1875.651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102867,2149712,"POLYGON ((1231136.200 3594505.992, 1231136.200...",0.0,1.231161e+06,3.594531e+06,平顶山公园,4.68,p_86,1.232440e+06,3.593442e+06,1.231211e+06,3.594581e+06,"(92.18, 70.711, 1680.042, 1675.868)",92.18,70.711,1680.042,1675.868
102868,2149713,"POLYGON ((1231186.200 3594505.992, 1231186.200...",0.0,1.231211e+06,3.594531e+06,平顶山公园,4.68,p_86,1.232440e+06,3.593442e+06,1.231211e+06,3.594581e+06,"(47.18, 50.0, 1642.291, 1675.868)",47.18,50.000,1642.291,1675.868
102869,2149714,"POLYGON ((1231236.200 3594505.992, 1231236.200...",0.0,1.231261e+06,3.594531e+06,平顶山公园,4.68,p_86,1.232440e+06,3.593442e+06,1.231211e+06,3.594581e+06,"(2.17, 70.711, 1605.21, 1675.868)",2.17,70.711,1605.210,1675.868
102870,2149719,"POLYGON ((1231486.200 3594505.992, 1231486.200...",0.0,1.231511e+06,3.594531e+06,平顶山公园,4.68,p_86,1.232440e+06,3.593442e+06,1.231461e+06,3.594581e+06,"(4.32, 70.711, 1431.641, 1502.141)",4.32,70.711,1431.641,1502.141


In [67]:
result_vlx = result_vlx[~pd.isna(result_vlx['angle'])]
result_vlx = result_vlx[['net_id','diff_building_grid','name','park_id','area','angle','50_larger','50_park','larger_park']]
result_vlx['diff_distance'] = result_vlx['50_park'] - result_vlx['larger_park']

In [68]:
result_vlx

,net_id,diff_building_grid,name,park_id,area,angle,50_larger,50_park,larger_park,diff_distance
0,1176133,0.0,九龙湖公园,p_53,12.97,144.38,100.000,1957.807,1875.651,82.156
1,1176990,0.0,九龙湖公园,p_53,12.97,99.38,70.711,1888.464,1875.651,12.813
2,1176991,0.0,九龙湖公园,p_53,12.97,144.38,50.000,1916.517,1875.651,40.866
3,1176992,0.0,九龙湖公园,p_53,12.97,170.62,70.711,1945.451,1875.651,69.800
4,1176993,0.0,九龙湖公园,p_53,12.97,152.19,111.803,1975.226,1875.651,99.575
...,...,...,...,...,...,...,...,...,...,...
102867,2149712,0.0,平顶山公园,p_86,4.68,92.18,70.711,1680.042,1675.868,4.174
102868,2149713,0.0,平顶山公园,p_86,4.68,47.18,50.000,1642.291,1675.868,-33.577
102869,2149714,0.0,平顶山公园,p_86,4.68,2.17,70.711,1605.210,1675.868,-70.658
102870,2149719,0.0,平顶山公园,p_86,4.68,4.32,70.711,1431.641,1502.141,-70.500


# supply by 1-2-3-4 parks

In [69]:
id_x = result_vlx.groupby(by=['net_id']).count().sort_values(by='park_id')['name'] # 57316

In [70]:
id_x.value_counts()

1     9792
2     6020
3     3295
4     2904
5     1503
7     1401
8     1305
6     1247
9      777
12     385
11     365
10     307
13     303
14      97
15      20
Name: name, dtype: int64

In [71]:
id1 = id_x[id_x == 1].index # 仅有1个公园提供服务
id2 = id_x[id_x == 2].index # 仅有1个公园提供服务
id3 = id_x[id_x == 3].index # 仅有1个公园提供服务
id4 = id_x[id_x == 4].index # 仅有1个公园提供服务

In [72]:
df_1 = result_vlx[result_vlx['net_id'].isin(id1)]
df_2 = result_vlx[result_vlx['net_id'].isin(id2)]
df_3 = result_vlx[result_vlx['net_id'].isin(id3)]
df_4 = result_vlx[result_vlx['net_id'].isin(id4)]
########################################

In [73]:
df_4.sort_values(by='net_id')

,net_id,diff_building_grid,name,park_id,area,angle,50_larger,50_park,larger_park,diff_distance
2358,1231451,2.547760,凤凰台公园（百家湖广场）,p_52,6.00,32.15,141.421,1659.939,1777.968,-118.029
2359,1231451,2.547760,杨家圩公园,p_50,16.59,14.80,141.421,758.599,894.466,-135.867
2360,1231451,2.547760,竹山公园,p_49,2.20,47.83,141.421,2019.472,2111.683,-92.211
2361,1231451,2.547760,竹山文化休闲公园,p_48,2.20,47.83,141.421,2019.472,2111.683,-92.211
2362,1231452,0.000000,凤凰台公园（百家湖广场）,p_52,6.00,50.58,111.803,1709.164,1777.968,-68.804
...,...,...,...,...,...,...,...,...,...,...
87347,1964885,0.539434,太平山公园,p_37,29.99,139.63,70.711,599.925,544.301,55.624
87350,1964886,-2.844179,文体公园,p_39,6.18,155.33,50.000,448.205,402.282,45.923
87349,1964886,-2.844179,聚宝山公园,p_116,113.85,0.51,50.000,2006.887,2056.885,-49.998
87351,1964886,-2.844179,太平山公园,p_37,29.99,175.36,50.000,594.151,544.301,49.850


In [74]:
sample = df_2[df_2['net_id'] == 1199231]
sample

,net_id,diff_building_grid,name,park_id,area,angle,50_larger,50_park,larger_park,diff_distance
1851,1199231,0.0,牛首山北坡公园,p_31,257.43,23.96,50.0,1684.584,1730.153,-45.569
1852,1199231,0.0,将军山公园,p_28,574.83,87.67,50.0,1985.016,1986.423,-1.407


In [75]:
sample.iloc[:1]

,net_id,diff_building_grid,name,park_id,area,angle,50_larger,50_park,larger_park,diff_distance
1851,1199231,0.0,牛首山北坡公园,p_31,257.43,23.96,50.0,1684.584,1730.153,-45.569


In [76]:
len(sample)

2

In [77]:
sample.iloc[1,2:]

name                将军山公园
park_id              p_28
area               574.83
angle               87.67
50_larger            50.0
50_park          1985.016
larger_park      1986.423
diff_distance      -1.407
Name: 1852, dtype: object

In [78]:
def split_data(x, n):
    x['diff_abs'] = x['diff_distance'].abs()
    x = x.sort_values(by='diff_abs')  # ascending=False, largest first.
    del x['diff_abs']
    df = x.iloc[:1]
    for i in range(1, n):
        sel = x.iloc[i,2:]
        df[f'park_id{i}'] = sel['park_id']
        df[f'name{i}'] = sel['name']
        df[f'area{i}'] = sel['area']
        df[f'angle{i}'] = sel['angle']
        df[f'50_larger{i}'] = sel['50_larger']
        df[f'50_park{i}'] = sel['50_park']
        df[f'larger_park{i}'] = sel['larger_park']
        df[f'diff_distance{i}'] = sel['diff_distance']
    
    return df

In [79]:
split_data(sample, 2)

,net_id,diff_building_grid,name,park_id,area,angle,50_larger,50_park,larger_park,diff_distance,park_id1,name1,area1,angle1,50_larger1,50_park1,larger_park1,diff_distance1
1852,1199231,0.0,将军山公园,p_28,574.83,87.67,50.0,1985.016,1986.423,-1.407,p_31,牛首山北坡公园,257.43,23.96,50.0,1684.584,1730.153,-45.569


In [80]:
df2 = df_2.groupby(by='net_id').apply(split_data, n=2)

In [81]:
df2.index.names = [1,2]

In [82]:
df2.reset_index()

,1,2,net_id,diff_building_grid,name,park_id,area,angle,50_larger,50_park,larger_park,diff_distance,park_id1,name1,area1,angle1,50_larger1,50_park1,larger_park1,diff_distance1
0,1199231,1852,1199231,0.000000,将军山公园,p_28,574.83,87.67,50.000,1985.016,1986.423,-1.407,p_31,牛首山北坡公园,257.43,23.96,50.000,1684.584,1730.153,-45.569
1,1199238,1854,1199238,0.000000,将军山公园,p_28,574.83,99.47,50.000,2020.995,2012.170,8.825,p_31,牛首山北坡公园,257.43,159.01,50.000,2007.998,1961.237,46.761
2,1200047,1855,1200047,0.000000,牛首山北坡公园,p_31,257.43,68.96,70.711,1706.045,1730.153,-24.108,p_28,将军山公园,574.83,42.67,70.711,1935.022,1986.423,-51.401
3,1200053,1857,1200053,0.000000,牛首山北坡公园,p_31,257.43,110.99,50.000,1979.701,1961.237,18.464,p_28,将军山公园,574.83,9.47,50.000,1962.868,2012.170,-49.302
4,1200054,1860,1200054,0.000000,将军山公园,p_28,574.83,54.47,70.711,1971.913,2012.170,-40.257,p_31,牛首山北坡公园,257.43,155.99,70.711,2026.036,1961.237,64.799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6015,2135008,102583,2135008,-0.891812,平顶山公园,p_86,4.68,103.54,111.803,529.323,491.874,37.449,p_87,太子山公园,20.26,140.85,111.803,1956.594,1868.620,87.974
6016,2135009,102585,2135009,-1.258947,平顶山公园,p_86,4.68,76.97,100.000,479.335,491.874,-12.539,p_87,太子山公园,20.26,167.41,100.000,1966.337,1868.620,97.717
6017,2135641,102587,2135641,0.000000,平顶山公园,p_86,4.68,110.11,111.803,1279.809,1237.058,42.751,p_87,太子山公园,20.26,35.95,111.803,2012.421,2101.856,-89.435
6018,2135650,102589,2135650,0.000000,平顶山公园,p_86,4.68,124.20,141.421,830.133,742.361,87.772,p_87,太子山公园,20.26,40.66,141.421,1974.542,2079.663,-105.121


In [83]:
"""
./Py/S01-variables.py to process all data.
"""

'\n./Py/S01-variables.py to process all data.\n'